### Import Libraries

In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from openpyxl import Workbook

### Segment Data
Get data in 10 segments which represent 15 minutes of data per segment.

In [55]:
filename = 'cD.csv'
size = 900

chunks = []

bigChedd = 0

for chunk in pd.read_csv(filename, chunksize=size):
    chunks.append(chunk)
    bigChedd +=1
    if bigChedd == 10: break

### Automate Data Aquisition

In [40]:
# define column names of interest
CHG_V_headerName = "CHG_VOut[V]"
CHG_I_headerName = "CHG_IOut[A]"
BAT_I_headerName = "BATT_Current_String_Min[Amps]"
#Heater_Iout = Charger_Iout - Battery_Iout

headers = {
    0 : CHG_V_headerName,
    1 : CHG_I_headerName,
    2 : BAT_I_headerName
    }

def get_time_and_data(column_headerName, df):
    column_number = df.columns.get_loc(column_headerName)
    data = df.iloc[:, column_number]
    time = df.iloc[:, column_number+1]
    return data, time

def header_cases(arg, df):
    header = headers.get(arg)
    return get_time_and_data(header, df)

def get_header_values(df):
    chunksDT = []
    for i in range(len(headers)):
        data, time = header_cases(i, df)
        temp = []
        if data.shape == time.shape:
            temp = [(d, t) for d, t in zip(data, time)]
        
        name_start_idx = headers[i].rfind(':')
        name = headers[i][name_start_idx+1:]
        chunksDT.append((name, temp))
        
    return chunksDT

def to_excel(wb, data, sheetNumber):
    colSkip = 1
    sheetName = f'Group{sheetNumber}'
    sheet = wb.create_sheet(title=sheetName) # Create and name a new sheet
    wb.active = wb[sheetName]
    for sheet_data in data:
        
        sheet.cell(row=1, column=colSkip, value=sheet_data[0])
        idx = sheet_data[0].rfind('[')
        sheet.cell(row=1, column=colSkip+1, value=f'{sheet_data[0][:idx]} Time[s]')
        
        for row_index, row_data in enumerate(sheet_data[1], start=2):
            for col_index, value in enumerate(row_data, start=colSkip):
                sheet.cell(row=row_index, column=col_index, value=value)
        colSkip += 2

### Create an Excel File and Output Data

In [58]:
wb = Workbook() # create workbook object

In [59]:
num = 1
for chunk_df in chunks:
    data = get_header_values(chunk_df)

    to_excel(wb, data, sheetNumber=num)
    num+=1
wb.save('output.xlsx')